# Knowledge Graph Builder Example

This notebook demonstrates the process of building a knowledge graph from DataFrames using the functions defined in `kg_builder.py`. It includes examples of how to extract entities and generate Cypher commands.

In [9]:
import sys
import os
import pandas as pd
from kg_builder import describe_tables, extract_entities_and_relations, build_cypher_commands, LocalLLM, GoogleAIStudioLLM
# Add the parser directory to sys.path
import pathlib
notebook_dir = pathlib.Path().resolve()
parser_dir = notebook_dir / '../parser'
sys.path.append(str(parser_dir.resolve()))

from parser import parse_xml

# Sample DataFrame
# Parse the XML file
xml_file_path = '../parser/data/RAN_CM_DATA_SAMPLES.xml'  # Adjust path as needed
dfs, metadata, metadata2 = parse_xml(xml_file_path)

In [12]:
# Step 1: Describe tables
import yaml
import os

CONFIG_PATH = os.path.join(notebook_dir, "../config.yaml")
with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)
GOOGLE_API_KEY = config["google_ai_studio"]["api_key2"]
GOOGLE_MODEL = config["google_ai_studio"].get("model", "gemini-1.5-flash-latest")

llm = GoogleAIStudioLLM(api_key=GOOGLE_API_KEY, model=GOOGLE_MODEL)
descriptions = describe_tables(dfs, llm)

In [ ]:
# Step 2: Extract entities & relationships
entities, relationships = extract_entities_and_relations(descriptions, llm)

In [ ]:
# Step 3: Build Cypher commands
cypher_cmds = build_cypher_commands(entities, relationships)

In [ ]:
# Step 4: Execute on Neo4j
execute_cypher(cypher_cmds)